# Importation des packages 

In [1]:
import numpy as np
import pandas as pd
import re
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


##Importation des packages textuelles

In [0]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
from keras.preprocessing.text import text_to_word_sequence

# Setup de la random seed

In [0]:
seed = 7
np.random.seed(seed)

## Charger le dataset Iris

In [0]:
dataframe = pd.read_csv("Iris.csv", header=None, skiprows=1)
dataset = dataframe.values
X = dataset[:,1:5].astype(float)
Y = dataset[:,5]
type(Y)

FileNotFoundError: ignored

In [0]:
dataframe.drop(columns=1, inplace=True)

In [0]:
dataframe.head()

,0
0,type
1,INFJ
2,ENTP
3,INTP
4,INTJ


##Charger le dataset Textuelle

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
dataframe = pd.read_csv("/content/drive/My Drive/mbti.csv", header=0)
dataset = dataframe.values
Y = dataset[:,0]
Ycat = np.unique(Y)
X = pd.DataFrame(data= dataset[:,1])

###déplacer MBTI vers le dossier racine

In [7]:
stopwords = {}
Ycat

array(['ENFJ', 'ENFP', 'ENTJ', 'ENTP', 'ESFJ', 'ESFP', 'ESTJ', 'ESTP',
       'INFJ', 'INFP', 'INTJ', 'INTP', 'ISFJ', 'ISFP', 'ISTJ', 'ISTP'],
      dtype=object)

###Preprocess

In [0]:
def preprocess(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'\|\|\|', ' ', x)
  x = re.sub(r'https?://(w{3})?(\.?\w+)+[\w?=\.\-/]+', '', x)
  x = re.sub(r'[^a-z\s]', '', x)                  # get rid of noise
  x = re.sub(r'\s{2,}', ' ', x)
  x = re.sub(r'^\s*', '', x)
  #x = [w for w in x.split() if w not in set(stopwords)]  # remove stopwords
  return x

In [0]:
Xprepro = X.T.apply(lambda x: preprocess(x.values))

decouper xprepro

In [0]:
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(
    Xprepro, Y, test_size=0.33, random_state=42)

In [15]:
X_train, X_test

(1860    my longtime best friend is self typed as an in...
 6394    its like i have very little social energy of m...
 5846    dear infj gal ive never thought ill fall in lo...
 7016    dont play robot with us your a tootsie pop ful...
 3653    you just succeeded in making my day d i think ...
                               ...                        
 5734    i have cats and a chihuahuapug mix its really ...
 5191    ever since i can remember i have sufferedlived...
 5390    ive known a couple of infj guys and they seem ...
 860     even the loner gets lonely i feel like its har...
 7270    before reading the responses to this thread i ...
 Length: 5812, dtype: object,
 2802    this when i lie its to avoid an unreasonable r...
 2166    i said zero flexibility and little time for da...
 1919    this has to be written with bias or something ...
 360     hauhuhauh you might be right muhicz im just wo...
 1115    her parents are kind of the go to college good...
                          

###Feature Engerinring

####ELI5 pip installer

In [0]:
pip install eli5

     |████████████████████████████████| 112kB 3.4MB/s 


####ELI5 import

In [0]:
import eli5
from eli5.lime import TextExplainer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline, make_pipeline

####ELI5

nouvelle feature 'vec' -> TFIDF (feature textuelle qui marche assez bien)
'svd' -> APC -> reduction de dimention (~ ne sert à rien !)

In [0]:
vec = TfidfVectorizer(min_df=3, stop_words='english',
                      ngram_range=(1, 2))
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
lsa = make_pipeline(vec, svd) #execute vec puis svd dans la pipeline (preprocessing)

clf = SVC(C=150, gamma=2e-2, probability=True) #classificateur SVC (support vetcor classification) -> model de classificateur parmis des milliers
pipe = make_pipeline(lsa, clf) #execute lsa puis ajoute le classificateur

In [0]:
pipe.fit(X_train, y_train) #go ! -> pretraitement des données -> application du classificateur sur les données
pipe.score(X_test, y_test) # affiche les résultats !!! Attention overfitting -> découper le set

0.6643381068808941

In [0]:
def print_prediction(doc):
    y_pred = pipe.predict_proba([doc])[0]
    for target, prob in zip(Ycat, y_pred):
        print("{:.3f} {}".format(prob, target))

doc = "I'm a fool. I like madness and little cats"
print_prediction(doc)

0.004 ENFJ
0.021 ENFP
0.008 ENTJ
0.038 ENTP
0.002 ESFJ
0.008 ESFP
0.003 ESTJ
0.003 ESTP
0.076 INFJ
0.290 INFP
0.217 INTJ
0.260 INTP
0.019 ISFJ
0.028 ISFP
0.010 ISTJ
0.013 ISTP


In [0]:
te = TextExplainer(random_state=42)
te.fit(doc, pipe.predict_proba)
te.show_prediction(target_names=Ycat.tolist())

In [0]:
te.metrics_

{'mean_KL_divergence': 0.23887338121194293, 'score': 0.7125134395638164}

####Suite

In [0]:
def predict_proba_len(docs):
    # nasty predict_proba - the result is based on document length,
    # and also on a presence of "medication"
    proba = [
        [0, 0, 1.0, 0] if len(doc) % 2 or 'medication' in doc else [1.0, 0, 0, 0]
        for doc in docs
    ]
    return np.array(proba)

te3 = TextExplainer().fit(doc, predict_proba_len)
te3.show_prediction(target_names=Ycat.tolist())

In [0]:
doc2 = re.sub(r'(it|the|banned for|all|each)', '', doc, flags=re.I)
print_prediction(doc2)

0.024 ENFJ
0.052 ENFP
0.005 ENTJ
0.050 ENTP
0.001 ESFJ
0.002 ESFP
0.001 ESTJ
0.002 ESTP
0.345 INFJ
0.072 INFP
0.231 INTJ
0.177 INTP
0.003 ISFJ
0.015 ISFP
0.004 ISTJ
0.016 ISTP


##### !!! Test

In [0]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(
    subset='train',
    categories=categories,
    shuffle=True,
    random_state=42,
    remove=('headers', 'footers'),
)

In [0]:
Ycat.tolist()

['ENFJ',
 'ENFP',
 'ENTJ',
 'ENTP',
 'ESFJ',
 'ESFP',
 'ESTJ',
 'ESTP',
 'INFJ',
 'INFP',
 'INTJ',
 'INTP',
 'ISFJ',
 'ISFP',
 'ISTJ',
 'ISTP']

### Tokenizer (not eli5)

In [0]:
tokenizer.fit_on_texts(str(x) for x in Xprepro[0:].values)
print(tokenizer.word_counts)

Output hidden; open in https://colab.research.google.com to view.

###Encode class

In [0]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [19]:
encoded_Y

array([ 8,  3, 11, ..., 11,  9,  9])

###convert to dummies

In [20]:
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Base line Model
exemple: 2 couches -> 4 neurones mid || 3 neurones output (car trois classes possibles)

---
"kernel_initializer" à la place de "init"

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(4, input_dim=4, kernel_initializer='normal',activation='relu'))
  model.add(Dense(3, kernel_initializer='normal', activation='sigmoid'))

  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model textuelle - test

In [0]:
def baseline_model():
  model = Sequential()
  model.add(Dense(12, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
  model.add(Dense(8, kernel_initializer='normal', activation='relu'))
  model.add(Dense(12, kernel_initializer='normal', activation='sigmoid'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

## Exec

In [16]:
dummy_y

NameError: ignored

In [0]:
estimator = KerasClassifier(build_fn = baseline_model, nb_epoch = 200, batch_size = 6, verbose = 0)
kfold = KFold(n_splits = 10, shuffle= True, random_state = seed)
results = cross_val_score(estimator, X, dummy_y, cv = kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








Accuracy: 33.33% (12.65%)
